In [1]:
from pathlib import Path

import petl as etl

In [2]:
ks2 = (
    etl
        .fromcsv('../data/dfes/ks2_regional_and_local_authority_2016_to_2024_provisional.csv')
)

In [3]:
ks2_reading_fields = ['time_period', 'geo_code', 'geo_name', 'pt_read_met_higher_standard', 'pt_read_met_expected_standard', 'pt_gps_met_expected_standard']

ks2_reading = etl.cat(
    ks2
        .selecteq('gender', 'Total')
        .selecteq('geographic_level', 'National')
        .rename({
            'country_code': 'geo_code',
            'country_name': 'geo_name',
        })
        .cut(*ks2_reading_fields),
    ks2
        .selecteq('gender', 'Total')
        .select(
            lambda r: r.geographic_level == 'Regional' and r.region_code == 'E12000001'
        )
        .rename({
            'region_code': 'geo_code',
            'region_name': 'geo_name',
        })
        .cut(*ks2_reading_fields),
    ks2
        .selecteq('gender', 'Total')
        .selecteq('new_la_code', 'E08000021')
        .rename({
            'new_la_code': 'geo_code',
            'la_name': 'geo_name',
        })
        .cut(*ks2_reading_fields)
).replaceall('x', None)

In [4]:
OUT = Path('../src/data/education/_data/')
OUT.mkdir(exist_ok=True, parents=True)

In [5]:
ks2_reading.tocsv(OUT / 'ks2_reading.csv')